In [1]:
!pwd

/Users/shashanksahoo/Documents/KIT_Projects_2022/ICA_project_2023/kit-docgcn-dla-service/notebooks


In [2]:
import os
os.chdir('../py-bottom-up-attention') 

In [3]:
!pwd | ls

GETTING_STARTED.md    demo                  projects
INSTALL.md            detectron2            requirements.txt
LICENSE               detectron2.egg-info   sample_inderence.json
MODEL_ZOO.md          dev                   setup.cfg
README.md             docker                setup.py
build                 docs                  tests
configs               original_README.md    tools
datasets              original_demo


In [4]:
# IMPORTING Bottom-up Attention with Detectron2 - PYTORCH 
import os
import io

import detectron2

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog

# import some common libraries
import numpy as np
import cv2
import torch

# Show the image in ipynb
from IPython.display import clear_output, Image, display
import PIL.Image
def showarray(a, fmt='jpg'):
    a = np.uint8(np.clip(a, 0, 255))
    f = io.BytesIO()
    PIL.Image.fromarray(a).save(f, fmt)
    display(Image(data=f.getvalue()))

In [7]:
cfg = get_cfg()
cfg.merge_from_file("configs/VG-Detection/faster_rcnn_R_101_C4_caffe.yaml")
cfg.MODEL.RPN.POST_NMS_TOPK_TEST = 300
cfg.MODEL.ROI_HEADS.NMS_THRESH_TEST = 0.6
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.2
# VG Weight
cfg.MODEL.WEIGHTS = "http://nlp.cs.unc.edu/models/faster_rcnn_from_caffe.pkl"
cfg.MODEL.DEVICE = "cpu"
predictor = DefaultPredictor(cfg)

Config 'configs/VG-Detection/faster_rcnn_R_101_C4_caffe.yaml' has no VERSION. Assuming it to be compatible with latest v2.


Modifications for VG in RPN (modeling/proposal_generator/rpn.py):
	Use hidden dim 512 instead fo the same dim as Res4 (1024).

Modifications for VG in RoI heads (modeling/roi_heads/roi_heads.py):
	1. Change the stride of conv1 and shortcut in Res5.Block1 from 2 to 1.
	2. Modifying all conv2 with (padding: 1 --> 2) and (dilation: 1 --> 2).
	For more details, please check 'https://github.com/peteanderson80/bottom-up-attention/blob/master/models/vg/ResNet-101/faster_rcnn_end2end_final/test.prototxt'.



/Users/shashanksahoo/.torch/iopath_cache is not accessible! Using /var/folders/rq/73_t64xn2zxf07vggynh53800000gq/T/iopath_cache instead!


In [19]:
#NUM_OBJECTS = 36

from torch import nn

from detectron2.modeling.postprocessing import detector_postprocess
from detectron2.modeling.roi_heads.fast_rcnn import FastRCNNOutputLayers, FastRCNNOutputs, fast_rcnn_inference_single_image
from detectron2.structures.boxes import Boxes
from detectron2.structures.instances import Instances

def doit(raw_image, raw_boxes):
        # Process Boxes
    raw_boxes = Boxes(torch.from_numpy(raw_boxes))
    # raw_boxes = Boxes(torch.from_numpy(raw_boxes).cuda())
    
    with torch.no_grad():
        raw_height, raw_width = raw_image.shape[:2]
        # print("Original image size: ", (raw_height, raw_width))
        
        # Preprocessing
        image = predictor.transform_gen.get_transform(raw_image).apply_image(raw_image)
        # print("Transformed image size: ", image.shape[:2])
        
        # Scale the box
        new_height, new_width = image.shape[:2]
        scale_x = 1. * new_width / raw_width
        scale_y = 1. * new_height / raw_height
        #print(scale_x, scale_y)
        boxes = raw_boxes.clone()
        boxes.scale(scale_x=scale_x, scale_y=scale_y)
        
        # ----
        image = torch.as_tensor(image.astype("float32").transpose(2, 0, 1))
        inputs = [{"image": image, "height": raw_height, "width": raw_width}]
        images = predictor.model.preprocess_image(inputs)
        
        # Run Backbone Res1-Res4
        features = predictor.model.backbone(images.tensor)
        
        # Run RoI head for each proposal (RoI Pooling + Res5)
        proposal_boxes = [boxes]
        features = [features[f] for f in predictor.model.roi_heads.in_features]
        box_features = predictor.model.roi_heads._shared_roi_transform(
            features, proposal_boxes
        )
        feature_pooled = box_features.mean(dim=[2, 3])  # pooled to 1x1
        # print('Pooled features size:', feature_pooled.shape)
        
        # Predict classes and boxes for each proposal.
        pred_class_logits, pred_proposal_deltas = predictor.model.roi_heads.box_predictor(feature_pooled)
        # print(pred_class_logits.shape)
        pred_class_prob = nn.functional.softmax(pred_class_logits, -1)
        pred_scores, pred_classes = pred_class_prob[..., :-1].max(-1)
        
        # Detectron2 Formatting (for visualization only)
        roi_features = feature_pooled
        instances = Instances(
            image_size=(raw_height, raw_width),
            pred_boxes=raw_boxes,
            scores=pred_scores,
            pred_classes=pred_classes
        )
        
        return instances, roi_features

In [20]:
import os
train_json_path = '../docbank_data/docbank_training_data/annotations/'
train_json_ls = sorted(os.listdir(train_json_path))
len(train_json_ls)

100096

In [22]:
os.listdir(train_json_path)[-1], sorted(os.listdir(train_json_path))[-1]

('111.tar_1507.05116.gz_orientations_level_9.json',
 '141.tar_1706.09493.gz_be11_10.json')

In [23]:
from tqdm import tqdm
# Generating the bbox 
tr_img_bbox_list = {}
import json
for i,name in enumerate(tqdm(train_json_ls[:20])):
      tr_img_bbox_list[name[:-5]] = []
      path = train_json_path+name
      with open(path) as f:
            json_list = json.load(f)
      # json_list = json_list['form']
      for j in range(len(json_list)):
        tr_img_bbox_list[name[:-5]].append(json_list[j]['box'])
# print(tr_img_bbox_list)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 119.70it/s]


In [24]:
len(tr_img_bbox_list)

20

In [25]:
train_image_path = '../docbank_data/docbank_training_data/images/'
train_image_ls = sorted(os.listdir(train_image_path))

In [26]:
train_image_ls[0]

'1.tar_1401.0001.gz_infoingames_without_metric_arxiv_0_ori.jpg'

In [27]:
import os
img_ls = sorted(os.listdir(train_image_path))
# print(len(img_ls))
# visual_features_train_carry = {}
file_name_list = []
for i in tqdm(range(len(img_ls[:20])), position=0, leave=True):
      path = train_image_path+img_ls[i]
      im = cv2.imread(path)
      instances, features = doit(im, np.array(tr_img_bbox_list[img_ls[i][:-8]]))
      visual_features = features.tolist()

      # print(len(visual_features))
      # visual_features_train_carry[img_ls[i][:-4]]=visual_features

      file_name = img_ls[i][:-4]
      file_name_list.append(file_name)
      path = '../docbank_data/visual_features/object_train2/'+file_name+'.json'
      with open(path,'w') as file_object:
        json.dump(visual_features,file_object)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [05:45<00:00, 17.28s/it]


In [42]:
import os
test_json_path = '../docbank_data/docbank_testing_data/annotations/'
test_json_ls = sorted(os.listdir(test_json_path))

In [43]:
# Generating the bbox 
te_img_bbox_list = {}
import json
for i,name in tqdm(enumerate(test_json_ls[:20])):
  te_img_bbox_list[name[:-5]] = []
  path = test_json_path+name
  with open(path) as f:
    json_list = json.load(f)
  # json_list = json_list['form']
  for j in range(len(json_list)):
    te_img_bbox_list[name[:-5]].append(json_list[j]['box'])
# print(te_img_bbox_list)

20it [00:00, 296.41it/s]


In [44]:
test_image_path = '../docbank_data/docbank_testing_data/images/'
test_image_ls = sorted(os.listdir(test_image_path))

In [45]:
img_ls = sorted(os.listdir(test_image_path))
visual_features_test_carry = {}
for i in tqdm(range(len(img_ls[:20])),position=0, leave=True):
  path = test_image_path+img_ls[i]
  im = cv2.imread(path)
  print(img_ls[i][:-8])
  instances, features = doit(im, np.array(te_img_bbox_list[img_ls[i][:-8]]))
  visual_features = features.tolist()
    
  # print(len(visual_features))
  visual_features_test_carry[img_ls[i][:-4]]=visual_features
  file_name = img_ls[i][:-4]
  path = '../docbank_data/visual_features/object_test2/'+file_name+'.json'
  with open(path,'w') as file_object:
    json.dump(visual_features,file_object)

  0%|                                                                                                                                                                                      | 0/20 [00:00<?, ?it/s]

195.tar_1610.03346.gz_hj_19


  5%|████████▋                                                                                                                                                                     | 1/20 [00:14<04:33, 14.39s/it]

195.tar_1610.03346.gz_hj_35


 10%|█████████████████▍                                                                                                                                                            | 2/20 [00:23<03:27, 11.54s/it]

...

 90%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                 | 18/20 [04:17<00:20, 10.03s/it]

195.tar_1610.03355.gz_Slow10_4


 95%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎        | 19/20 [04:37<00:12, 12.88s/it]

195.tar_1610.03355.gz_Slow10_7


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [04:53<00:00, 14.66s/it]
